In [1]:
from langchain.llms  import GooglePalm

llm = GooglePalm()

c:\Users\AMIR1\Documents\ml\llm-project\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
llm.generate(["Hello World"])

LLMResult(generations=[[Generation(text='!')]], llm_output=None, run=[RunInfo(run_id=UUID('4fe1ecd6-6106-4be3-9ba5-139693faa98c'))])

In [3]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [4]:
from dotenv import dotenv_values

config = dotenv_values(".env")
db_user = config["PG_USER"]
db_pwd = config["PG_PASSWORD"]
db_host = config["PG_HOST"]
db_name = config["DB_NAME"]

db = SQLDatabase.from_uri(f"postgresql://{db_user}:{db_pwd}@{db_host}/{db_name}")

In [5]:
print(db.table_info)


CREATE TABLE discounts (
	discount_id SERIAL NOT NULL, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount NUMERIC(5, 2), 
	CONSTRAINT discounts_pkey PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_t_shirt_id_fkey FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_pct_discount_check CHECK (pct_discount >= 0::numeric AND pct_discount <= 100::numeric)
)

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
11	3863415	10.00
12	3863416	15.00
13	3863417	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id SERIAL NOT NULL, 
	brand brand_type NOT NULL, 
	color color_type NOT NULL, 
	size size_type NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	CONSTRAINT t_shirts_pkey PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_brand_color_size_key UNIQUE (brand, color, size), 
	CONSTRAINT t_shirts_price_check CHECK (price >= 10 AND price <= 50)
)

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock_quantity
3863415	Adidas	Black	XL	13

Query database with llm

In [6]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qns1 = db_chain("How many t-shirts do we have left for nike in extra small size and white color?")



> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for nike in extra small size and white color?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'
SQLResult: [(93,)]
Answer:93
> Finished chain.


In [7]:
qns2 = db_chain.run("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT SUM(price) 
FROM t_shirts 
WHERE size = 'S';
SQLResult: [(136,)]
Answer:136
> Finished chain.


In [8]:
sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """

qns3 = db_chain.run(sql_code)



> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery:select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('8614.10000000000000000000'),)]
Answer:8614.1
> Finished chain.


In [9]:
qns4 = db_chain.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(9587,)]
Answer:9587
> Finished chain.


In [10]:
qns5 = db_chain.run("How many white color Levi's t shirts we have available?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's t shirts we have available?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE color = 'White' AND brand = 'Levi'
SQLResult: [(91,)]
Answer:91
> Finished chain.


In [11]:
few_shots = [
    {'question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'sql_query' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'sql_result': "Result of the SQL query",
     'answer' : qns1["result"]},
    {'question': "How much is the total price of the inventory for all S-size t-shirts?",
     'sql_query':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'sql_result': "Result of the SQL query",
     'answer': qns2},
    {'question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'sql_query' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'sql_result': "Result of the SQL query",
     'answer': qns3} ,
     {'question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'sql_query': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'sql_result': "Result of the SQL query",
      'answer' : qns4},
    {'question': "How many white color Levi's shirt I have?",
     'sql_query' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'sql_result': "Result of the SQL query",
     'answer' : qns5
     }
]

In [58]:
few_shots

[{'question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'sql_query': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'sql_result': 'Result of the SQL query',
  'answer': '93'},
 {'question': 'How much is the total price of the inventory for all S-size t-shirts?',
  'sql_query': "SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
  'sql_result': 'Result of the SQL query',
  'answer': '136'},
 {'question': 'If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?',
  'sql_query': "SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from\n(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'\ngroup by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id\n ",
  'sql_result': 'Result of the SQL query',
  'answer

## Creating Semantic Similarity Based example selector
- Create embedding on the few_shots
- Store the embedding in a database (Chroma DB)
- Retrieve the top most semantically similar example from the database

In [12]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma


embeddings_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [13]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=few_shots,
    embeddings=embeddings_model,
    vectorstore_cls=Chroma,
    k=2
)

In [14]:
example_selector.select_examples({"Question": "How many Adidas T shirts I have left in my store?"})

[{'answer': '93',
  'question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'sql_query': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'sql_result': 'Result of the SQL query'},
 {'answer': '9587',
  'question': 'If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?',
  'sql_query': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
  'sql_result': 'Result of the SQL query'}]

## Prompt

In [15]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt
from langchain.prompts.prompt import PromptTemplate

print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [16]:
PROMPT_SUFFIX

'Only use the following tables:\n{table_info}\n\nQuestion: {input}'

In [40]:
example_prompt = PromptTemplate(
    input_variables=["answer", "question", "sql_query","sql_result"],
    template="Answer: {answer}\nQuestion: {question}\nSQLQuery: {sql_query}\nSQLResult: {sql_result}"
)

In [50]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=_mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

In [42]:
print(few_shot_prompt.format(input="How many white color Adidas shirt I have?", 
                             top_k="2", 
                             table_info=db.table_info))

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 2 results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the S

## Predict

In [53]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt, top_k=2)

In [44]:
new_chain("How many white color Levi's shirt I have?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's shirt I have?
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(91,)]
Answer:91
> Finished chain.


{'query': "How many white color Levi's shirt I have?", 'result': '91'}

In [45]:
new_chain.run("How many t-shirts do we have left for Nike in XS size and white color?")



> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for Nike in XS size and white color?
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'
SQLResult: [(93,)]
Answer:93
> Finished chain.


'93'

In [46]:
new_chain("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT price*stock_quantity FROM t_shirts WHERE size = 'S'
SQLResult: [(1548,), (3350,), (2912,), (1404,)]
Answer:9214
> Finished chain.


{'query': 'How much is the price of the inventory for all small size t-shirts?',
 'result': '9214'}

In [55]:
a_input = "If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?"

print(few_shot_prompt.format(input=a_input, 
                             top_k="2", 
                             table_info=db.table_info))

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 2 results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the S

In [57]:
new_chain(a_input)



> Entering new SQLDatabaseChain chain...
If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?
SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Nike'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('11745.20000000000000000000'),)]
Answer:11745.2
> Finished chain.


{'query': 'If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?',
 'result': '11745.2'}